<a href="https://colab.research.google.com/github/ronaldgenesmith3/Portfolio/blob/main/Italian_Vacation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Italian Vacation for Gelato Lovers

## Table of contents
* [Introduction: Business Problem](#introduction)
* [Data](#data)
* [Analysis](#analysis)
* [Conclusion](#conclusion)

## Introduction: Business Problem <a name="introduction"></a>



In this project we will try to find an optimal location for a gelato lovers Itailian vacation. The stakeholders for this project would be anyone who loves gelato and wants to visit Italy.

We will focus on finding which of Italy's more famous destinations would be best for any gelato lover. We are particulary interested in the number of gelato shops as well as the average distance to gelato shops.

We will use our data science powers to generate a few most promissing destinations based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.

## Data <a name="data"></a>

Based on definition of our problem, factors that will influence our decission are:
* number of gelato shops in the city.
* average distance of shops from the center of all locations

I decided to use four square API in order to find the locations of all gelato shops in the city.

Following data sources will be needed to extract/generate the required information:
* Latitude and Longitude coordiantes of each shop 
* Number of locations in each city
* Coordinate of the center of all gelato shops in each city.

### Neighborhood Candidates

Four of the most popular destinations in Italy are Rome, Florence, Venice, and Naples. We will focus on these four cities for our discussion.

## Analysis <a name="analysis"></a>

Let's do some basic exploratory data analysis to discover the best city for our gelato lover's vacation.

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
!pip install folium
import folium# map rendering library

print('Libraries imported.')

Libraries imported.


In [ ]:
#Load Four Square API Credentials
CLIENT_ID = 'DUVVXNDAU4QPONXXUN4S1TAOFSER3PZTT4CHE23USXHULRR1' # your Foursquare ID
CLIENT_SECRET = 'E4ITWCAHSNRTP4Z1Z0BFZDT3UGQ1CG51Q24ZLHSYJCGR0MLP' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)

Your credentails:
CLIENT_ID: DUVVXNDAU4QPONXXUN4S1TAOFSER3PZTT4CHE23USXHULRR1


In [ ]:
#Use Four Square API to find the 100 nearest gelato places in each city
LIMIT = 500 # Maximum is 100
cities = ["Rome, IT", 'Florence, IT', 'Venice, IT', 'Naples, IT']
results = {}
for city in cities:
    url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&near={}&limit={}&categoryId={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        VERSION, 
        city,
        LIMIT,
        "4bf58dd8d48988d1c9941735") # Ice Cream CATEGORY ID
    results[city] = requests.get(url).json()

In [ ]:
#Create a dataframe to store the information from four square.
df_venues={}
for city in cities:
    venues = pd.json_normalize(results[city]['response']['groups'][0]['items'])
    df_venues[city] = venues[['venue.name', 'venue.location.address', 'venue.location.lat', 'venue.location.lng']]
    df_venues[city].columns = ['Name', 'Address', 'Lat', 'Lng']

In [ ]:
#make use the dataframe to find the number of gelato stores in each city, and map their locations to be able to visually analyze each city.
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)

    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])  
    print(f"Total number of gelato places in {city} = ", results[city]['response']['totalResults'])
    print("Showing Top 100")

Total number of gelato places in Rome, IT =  138
Showing Top 100
Total number of gelato places in Florence, IT =  89
Showing Top 100
Total number of gelato places in Venice, IT =  50
Showing Top 100
Total number of gelato places in Naples, IT =  42
Showing Top 100


In [ ]:
#Map of Rome's gelato locations
maps[cities[0]]

In [ ]:
#Map of Florence Gelato locations
maps[cities[1]]

In [ ]:
#Map of Vience's Gelato Locations
maps[cities[2]]

In [ ]:
#Map of Naples Gelato Locations
maps[cities[3]]

In [ ]:
#Find the center of all of the gelato locations for each city and then determine the average distance from that center to each of the gelato locations.
maps = {}
for city in cities:
    city_lat = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lat'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lat']])
    city_lng = np.mean([results[city]['response']['geocode']['geometry']['bounds']['ne']['lng'],
                        results[city]['response']['geocode']['geometry']['bounds']['sw']['lng']])
    maps[city] = folium.Map(location=[city_lat, city_lng], zoom_start=11)
    venues_mean_coor = [df_venues[city]['Lat'].mean(), df_venues[city]['Lng'].mean()] 
    # add markers to map
    for lat, lng, label in zip(df_venues[city]['Lat'], df_venues[city]['Lng'], df_venues[city]['Name']):
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(maps[city])
        folium.PolyLine([venues_mean_coor, [lat, lng]], color="green", weight=1.5, opacity=0.5).add_to(maps[city])
    
    label = folium.Popup("Mean Co-ordinate", parse_html=True)
    folium.CircleMarker(
        venues_mean_coor,
        radius=10,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(maps[city])

    print(city)
    print("Mean Distance from Mean coordinates")
    print(np.mean(np.apply_along_axis(lambda x: np.linalg.norm(x - venues_mean_coor),1,df_venues[city][['Lat','Lng']].values)))

Rome, IT
Mean Distance from Mean coordinates
0.02499697176159012
Florence, IT
Mean Distance from Mean coordinates
0.009850202668789741
Venice, IT
Mean Distance from Mean coordinates
0.0195955977264324
Naples, IT
Mean Distance from Mean coordinates
0.01869192356821415


In [ ]:
#Visual representation of the center of Rome
maps[cities[0]]

In [ ]:
#Visual Representation of Florence
maps[cities[1]]

In [ ]:
#Visual representation of Venice
maps[cities[2]]

In [ ]:
#Visual representation of Naples
maps[cities[3]]

## Conclusion <a name="conclusion"></a>

##If your preference is most locations then Rome is where you should go.

Rome has a total of 138 gelato places. This is almost 50 more than second place which is Florence at 89. Venice is third with 50 and Naples is last with 42.

This is probably not the best way to find the best place to go as it would take you over six and a half weeks to try every one of the gelato places in Rome if you had three different places a day.

##What about distance?

So you have decided to not only factor how many, but also the distance. Chances are if you are so obsessed with ice cream, then you probably don't want to walk very far to get some.

Florence came in first with an average distance of 0.0099 as the average distance from the center of all locations. Second was Naples with 0.019, third was Venice with 0.020, and fourth was Rome with 0.025.

This was not even close. Florence was by far the best city for the average distance from the center to the gelato places.

##So where should I go?

Based on the data I would choose to go to Florence. You have plenty of gelato plaes to try with almost 90. The average distance is only 0.0099 so you are very close and within walking distance of most places.

Using the map above we can actually find where would be the best place to look for a place to stay. You would want to try and stay as close to the center of the map as possible. This location is actually right beside the most famous landmark of Florence, the duomo.

##What Next?

Now that you know where the best place to go is Florence all you have to do is find a hotel and a plane ticket.

##Enjoy your trip